### Retreive Venue ids from CSV, to eventually populate db

In [2]:
import pandas as pd

In [24]:
df = pd.read_csv('./gap_venues.csv', index_col = 0)

In [25]:
df.shape

(342, 2)

In [32]:
first_venue = venues[0]

In [28]:
import requests
url = "https://data.texas.gov/resource/naix-2893.json"

In [70]:
def build_end_date(year, month):
    return {'obligation_end_date_yyyymmdd': f'{year}-{month}-31T00:00:00.000'}

# params = build_end_date(2020, 1)
# params.update(first_venue)
# response = requests.get(url, params)
# response.json()

In [ ]:
venue = ['location_name', 'taxpayer_name', 'location_number']
location = ['city', 'state', 'zip_code', 'location_address']
keys = ['wine_receipts', 'beer_receipts', 'total_receipts']

In [16]:
import sqlite3
conn = sqlite3.connect('restaurants.db')
cursor = conn.cursor()

In [20]:
import requests
url = "https://data.texas.gov/resource/naix-2893.json"
params = {'obligation_end_date_yyyymmdd': '2020-1-31T00:00:00.000'}

In [36]:
venue_ids = df.to_dict('records')

In [40]:
len(venue_ids)

342

### 

In [34]:
venues = []
for venue_id in venue_ids:
    params = {'obligation_end_date_yyyymmdd': '2020-1-31T00:00:00.000'}
    params.update(venue_id)
    response = requests.get(url, params)
    venues.append(response.json()[0])

In [36]:
def reduced_venues():
    venue_identifiers = ['location_name', 'taxpayer_number', 'location_number', 'responsibility_begin_date_yyyymmdd']
    location = ['location_city', 'location_zip', 'location_address']
    selected_attrs = venue_identifiers + location
    scoped_venues = []
    for venue in venues:
        scoped_venues.append({k:v for k, v in venue.items() if k in selected_attrs})
    return scoped_venues

In [46]:
scoped_vens = reduced_venues()

In [1]:
# scoped_vens[:3]

In [10]:
import pandas as pd 

In [12]:
# df_scoped = pd.DataFrame(scoped_vens)

In [13]:
# df_scoped.to_sql('venues',conn, index=True, index_label='id')

### Select venues from sql

In [71]:
sql_venues = pd.read_sql('select * from venues;', conn)

In [77]:
sql_vens = sql_venues.to_dict('records')

In [79]:
venue = sql_vens[0]
receipts = find_receipts_by_number(venue, months = 1)
receipts[0].keys()

dict_keys(['taxpayer_number', 'taxpayer_name', 'taxpayer_address', 'taxpayer_city', 'taxpayer_state', 'taxpayer_zip', 'taxpayer_county', 'location_number', 'location_name', 'location_address', 'location_city', 'location_state', 'location_zip', 'location_county', 'inside_outside_city_limits_code_y_n', 'tabc_permit_number', 'responsibility_begin_date_yyyymmdd', 'obligation_end_date_yyyymmdd', 'liquor_receipts', 'wine_receipts', 'beer_receipts', 'cover_charge_receipts', 'total_receipts'])

In [81]:
def find_begin_dates(venues):
    begin_dates = []
    for venue in sql_vens:
        receipts = find_receipts_by_number(venue, months = 1)
        first_receipt = receipts[0]
        begin_dates.append(first_receipt['responsibility_begin_date_yyyymmdd'])

In [83]:
len(begin_dates)

342

In [84]:
begin_date = pd.Series(begin_dates)

In [86]:
sql_venues['begin_date'] = begin_date

In [92]:
# find_receipts_by_number(sql_vens[1], months = 1)

,id,location_address,location_city,location_name,location_number,location_zip,taxpayer_number,begin_date
0,0,12121 WESTHEIMER RD STE 130,HOUSTON,PELAZZIO BANQUETS CATERING AND CONFERENCES,1,77077,17606321739,2000-05-09T00:00:00.000
1,1,21919 INVERNESS FOREST BLVD,HOUSTON,ASHTON GARDENS,1,77073,10207051466,2005-01-25T00:00:00.000


In [48]:
def find_receipts_by_number(venue, order_by = 'obligation_end_date_yyyymmdd DESC', months = 36):
    taxpayer_number = venue['taxpayer_number']
    response = requests.get(url,
                            {'taxpayer_number': taxpayer_number, 
                             '$ORDER': order_by})
    receipts = response.json()[:months]
    return receipts

In [53]:
def transform_receipts(receipts,  venue):
    scoped_receipts = [scope_receipt(receipt) for receipt in receipts]
    for receipt in scoped_receipts:
        receipt['venue_id'] = venue['id']
    return scoped_receipts

In [5]:
def scope_receipt(receipt, attrs = ['obligation_end_date_yyyymmdd', 'liquor_receipts', 'wine_receipts', 'beer_receipts', 'cover_charge_receipts', 'total_receipts']):
    return {k:v for k,v in receipt.items() if k in attrs}

In [56]:
def collect_receipts():
    receipts_dfs = []
    for venue in sql_vens:
        receipts = find_receipts_by_number(venue)
        scoped_receipts = transform_receipts(receipts,  venue)
        df = pd.DataFrame(scoped_receipts)
        receipts_dfs.append(df)
    return receipts_dfs

In [58]:
for receipts_df in receipts_dfs:
    receipts_df.to_sql('receipts',conn, if_exists = 'append')

In [61]:
pd.read_sql('select count(*) from receipts;', conn)

,count(*)
0,11830


,index,beer_receipts,cover_charge_receipts,liquor_receipts,obligation_end_date_yyyymmdd,total_receipts,venue_id,wine_receipts
0,0,4505,0,14641,2020-01-31T00:00:00.000,22525,0,3379


In [63]:
pd.read_sql('select COUNT(DISTINCT(venue_id)) from receipts;', conn)

,COUNT(DISTINCT(venue_id))
0,342


In [103]:
sql_venues.to_sql('venues', index = False, if_exists = 'replace', con = conn)

In [98]:
receipts_df = pd.read_sql('select * from receipts;', conn)

In [115]:
# receipts_df.loc[:, 'beer_receipts':].to_sql('receipts', if_exists = 'replace', index_label = 'id', con = conn)

In [116]:
# receipts_df.to_sql('receipts', index_label = 'id', if_exists = 'replace', con = conn)

In [117]:
pd.read_sql('select * from receipts limit 1;', conn)

,id,beer_receipts,cover_charge_receipts,liquor_receipts,obligation_end_date_yyyymmdd,total_receipts,venue_id,wine_receipts
0,0,4505,0,14641,2020-01-31T00:00:00.000,22525,0,3379


In [96]:
pd.read_sql('select * from venues limit 1;', conn)

,id,location_address,location_city,location_name,location_number,location_zip,taxpayer_number,begin_date
0,0,12121 WESTHEIMER RD STE 130,HOUSTON,PELAZZIO BANQUETS CATERING AND CONFERENCES,1,77077,17606321739,2000-05-09T00:00:00.000


In [69]:
# pd.read_sql('SELECT venue_id, COUNT(*) as receipts_per_ven FROM receipts GROUP BY venue_id ORDER BY receipts_per_ven ASC LIMIT 10', conn)